# RDF-centered conversion
This notebook will convert ZIB triples to capacity fields.

In [1]:
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import RDF
from pathlib import Path

## ZIB ontology
A small sample ontology to be able to describe ZIB concepts

In [5]:
zib_ontology = '../../ZIB/zib.owl'
zib = Namespace('http://example.org/ZIB')
zib_ontology = Graph(identifier=zib_namespace).parse(zib_ontology)


In [12]:
# Sample medicationUse zib
patient_graph = Graph()
patient_graph.bind('zib', zib_ontology.identifier)
patient_graph.bind('ATC', 'http://purl.bioontology.org/ontology/ATC')

patient = Namespace('http://example.org/patient/')
zib_record = Namespace('http://example.org/zib_record/')

patient_graph.add((patient.bob, URIRef('zib#hasZibRecord'), zib_record.bobsmedication))
patient_graph.add((zib_record.bobsmedication, RDF.type, zib.MedicationUse))
patient_graph.add((zib_record.bobsmedication, URIRef('zib#medicationCode'), URIRef('ATC#C01BA01')))

print(patient_graph.serialize('zibpatient.ttl',format='turtle'))


None
